In [ ]:
import numpy as np
import scipy.signal
import plotly.graph_objects as go


In [ ]:
lower_freq = 500
upper_freq = 5000
T = 2
fs = 192000


phase_rate = T / np.log(upper_freq / lower_freq)
f = np.fft.rfftfreq(T * fs, 1 / fs)
t = np.arange(T * fs) / fs

time_signal = np.sin(2 * np.pi * lower_freq * phase_rate * np.exp(t / phase_rate))

# time_signal = np.sin(
#     (2 * np.pi * lower_freq * T)
#     /
#     np.log(upper_freq / lower_freq)
#     * np.exp(
#         t / T * np.log(upper_freq / lower_freq)
#     ))
direct_spectrum = 0.5 * (phase_rate / f) ** 0.5 * np.exp(2j * np.pi * f * phase_rate * (1 - np.log(f / lower_freq)) - 1j * np.pi / 4)
inverse_spectrum = 2 * (f / phase_rate) ** 0.5 * np.exp(-2j * np.pi * f * phase_rate * (1 - np.log(f / lower_freq)) + 1j * np.pi / 4)
inverse_spectrum[0] = 0
direct_spectrum[0] = 0
direct_signal = np.fft.irfft(direct_spectrum)
inverse_signal = np.fft.irfft(inverse_spectrum)


In [ ]:
f_spec, t_spec, Sxx = scipy.signal.spectrogram(direct_signal, fs=fs, window='hann', nperseg=2**15)
Sdb = 10*np.log10(np.abs(Sxx))
go.Figure(go.Heatmap(x=t_spec, y=f_spec, z=Sdb, zmax=Sdb.max(), zmin=Sdb.max() - 60)).update_yaxes(type='log').show()

f_spec, t_spec, Sxx = scipy.signal.spectrogram(inverse_signal, fs=fs, window='hann', nperseg=2**15)
Sdb = 10*np.log10(np.abs(Sxx))
go.Figure(go.Heatmap(x=t_spec, y=f_spec, z=Sdb, zmax=Sdb.max(), zmin=Sdb.max() - 60)).update_yaxes(type='log').show()

f_spec, t_spec, Sxx = scipy.signal.spectrogram(time_signal, fs=fs, window='hann', nperseg=2**15)
Sdb = 10*np.log10(np.abs(Sxx))
go.Figure(go.Heatmap(x=t_spec, y=f_spec, z=Sdb, zmax=Sdb.max(), zmin=Sdb.max() - 60)).update_yaxes(type='log').show()

go.Figure([
    go.Scatter(x=f, y=20*np.log10(np.abs(direct_spectrum))),
    go.Scatter(x=f, y=20*np.log10(np.abs(np.fft.rfft(time_signal)))),
]).update_xaxes(type='log').show()


In [ ]:
scipy.signal.iir


In [ ]:

sos = scipy.signal.iirfilter(N=8, Wn=5000, fs=fs, btype='lowpass', output='sos')
response = scipy.signal.sosfilt(sos, time_signal)

response = response + response ** 2 + response **3 #+ response ** 4 + response ** 5 + response ** 21

f_spec, t_spec, Sxx = scipy.signal.spectrogram(response, fs=fs, window='hann', nperseg=2**15)
Sdb = 10*np.log10(np.abs(Sxx))
go.Figure(go.Heatmap(x=t_spec, y=f_spec, z=Sdb, zmax=Sdb.max(), zmin=Sdb.max() - 60)).update_yaxes(type='log').show()

impulse = np.fft.irfft(
    np.fft.rfft(response, response.size * 2)
    *
    np.fft.rfft(inverse_signal, response.size * 2)
    # np.fft.rfft(time_signal, response.size).conj()
)

# impulse = np.fft.irfft(np.fft.rfft(time_signal) * spectrum)
go.Figure(go.Scatter(x=np.arange(impulse.size) / fs, y=impulse)).show()

f_spec, t_spec, Sxx = scipy.signal.spectrogram(impulse, fs=fs, window='hann', nperseg=2**15)
Sdb = 10*np.log10(np.abs(Sxx))
go.Figure(go.Heatmap(x=t_spec, y=f_spec, z=Sdb, zmax=Sdb.max(), zmin=Sdb.max() - 60)).update_yaxes(type='log').show()
